In [0]:
#Importación de librerías
import pandas as pd
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta
from unicodedata import normalize
import re
import time
import os
import handyspark as hpys
import mlflow
import mlflow.sklearn
import mlflow.spark

import pyspark.sql.functions as F
from pyspark.sql.functions import expr, col, column, monotonically_increasing_id, desc, when, length,isnan,count, monotonically_increasing_id, row_number, explode
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, LongType, BooleanType, DoubleType, TimestampType
from pyspark.sql import Window
from pyspark.ml import Pipeline as Pipeline_spark
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import contexto
from contexto.limpieza import *
import sparknlp
from sparknlp.annotator import *
from sparknlp.annotator import Tokenizer as Tokenizer_sparknlp
from sparknlp.annotator import Normalizer as Normalizer_sparknlp
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.training import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM, SpatialDropout1D, Embedding, Input, Bidirectional, BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing import sequence
from keras.layers.merge import concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import FalseNegatives, FalsePositives, TrueNegatives, TruePositives, Precision, Recall 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks, regularizers

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.metrics import precision_score, confusion_matrix

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import contexto
from contexto.lectura import leer_texto
from contexto.limpieza import limpieza_texto, lista_stopwords, remover_stopwords
from contexto.exploracion import grafica_barchart_frecuencias, obtener_ngramas, par_nubes

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [0]:
# storage_account = "blobName"
# container = "data"
# blobKey = "blobKey"
# blobEndpoint = "wasbs://{1}@{0}.blob.core.windows.net/".format(storage_account, container)

# try:
#   dbutils.fs.mount(
#     source = blobEndpoint,
#     mount_point = "/mnt/data",
#     extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})
# except:
#   print("Already mounted. Unmounting and trying to mount again")
#   dbutils.fs.unmount(mount_point='/mnt/data')
#   dbutils.fs.mount(
#     source = blobEndpoint,
#     mount_point = "/mnt/data",
#     extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})

In [0]:
# storage_account = "blobName"
# container = "basecruzada"
# blobKey = "blobKey"
# blobEndpoint = "wasbs://{1}@{0}.blob.core.windows.net/".format(storage_account, container)

# try:
#   dbutils.fs.mount(
#     source = blobEndpoint,
#     mount_point = "/mnt/basecruzada",
#     extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})
# except:
#   print("Already mounted. Unmounting and trying to mount again")
#   dbutils.fs.unmount(mount_point='/mnt/cruzada')
#   dbutils.fs.mount(
#     source = blobEndpoint,
#     mount_point = "/mnt/basecruzada",
#     extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})

In [0]:
# ## Importación de base de trabajo resultado de script cruce_limpieza
# storage_account = "blobName"
# container = "newdata"
# blobKey = "blobKey"
# blobEndpoint = "wasbs://{1}@{0}.blob.core.windows.net/".format(storage_account, container)

# try:
#   dbutils.fs.mount(
#     source = blobEndpoint,
#     mount_point = "/mnt/newdata",
#     extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})
# except:
#   print("Already mounted. Unmounting and trying to mount again")
#   dbutils.fs.unmount(mount_point='/mnt/newdata')
#   dbutils.fs.mount(
#     source = blobEndpoint,
#     mount_point = "/mnt/newdata",
#     extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})

# Creación de "nuevos" datos (desde junio 2020 hasta mayo 2021) para ejemplificar actualización diaria de registros

In [0]:
# Lectura de datos completos y selección de solo registros que vienen en peticiones y son empleados para predicción
file = "/mnt/cruzada/basecruzada.parquet"
new_data = spark.read.parquet(file)

columnas_peticiones = ['NumeroPeticion', 'EsPeticionHija', 'fechaRegistroPeticion', 'AnoRegistroPeticion', 'MesRegistroPeticion', 'TipoPeticion', 'MotivoPeticion', 'RegionalCreacion', 'CZCreacion', 'RegionalDireccionada', 'CZDireccionado', 'CanalRecepcion', 'TipoDocumentoPeticionario', 'NumeroDocPeticionario', 'NombresApellidosPeticionario', 'PrimerNombrePeticionario', 'SegundoNombrePeticionario', 'PrimerApellidoPeticionario', 'SegundoApellidoPeticionario', 'DepartamentoPeticionario', 'MunicipioPeticionario', 'CodDptoPeticionario', 'CodMunicipioPeticionario', 'BarrioVeredaPeticionario', 'DireccionPeticionario', 'SenalesParticularesUbicacionPeticonarioRural', 'DetalleTipoZonaPeticionario', 'DetalleZonaPeticionario', 'EdadPeticionario', 'SexoPeticionario', 'GrupoEtnicoPeticionario', 'PuebloIndigenaPeticionario', 'PresentaDiscapacidadPeticionario', 'CualDiscapacidadPeticionario', 'CondicionDesplazamientoPeticionario', 'HaSidoVictimaPeticionario', 'EstadoPeticion', 'TipoDocumentoAfectado', 'NumeroDocAfectado', 'NombresApellidosAfectado', 'PrimerNombreAfectado', 'SegundoNombreAfectado', 'PrimerApellidoAfectado', 'SegundoApellidoAfectado', 'DepartamentoAfectado', 'MunicipioAfectado', 'CodDptoAfectado', 'CodMunicipioAfectado', 'BarrioVeredaAfectado', 'DireccionAfectado', 'ObsParticUbicaAfectado', 'DetalleTipoZonAfectado', 'DetalleZonAfectado', 'FechaNacAfectado', 'EdadAfectado', 'TipoEdadAfectado', 'SexoAfectado', 'GrupoEtnicoAfectado', 'PuebloIndigenaAfectado', 'PresentaDiscapacidadAfectado', 'CualDiscapacidadAfectado', 'CondicionDesplazamientoAfectado', 'PrimerMotivoIngreso', 'UltimoMotivoIngreso', 'TipoPeticionario', 'CategorizacionPRD_165', 'TipoDemandado', 'D_CONSTATADAS', 'CONSTATACION', 'PaisResidenciaPeticionario', 'PaisResidenciaAfectado', 'DescripcionPeticion']

new_data = new_data.select(columnas_peticiones)

In [0]:
# "Nuevos" datos de registros Junio y Julio de 2020 que no fueron incluidos en entrenamiento, pero serán utilizados para mostrar los resultados en el tablero
print((new_data.count(), len(new_data.columns)))
new_data = new_data.filter((col("AnoRegistroPeticion") == "2021"))
print((new_data.count(), len(new_data.columns)))

(1303161, 72)
(85696, 72)

In [0]:
new_data.groupby("AnoRegistroPeticion", "MesRegistroPeticion").count().show(200, truncate=False)

+-------------------+-------------------+-----+
AnoRegistroPeticion|MesRegistroPeticion|count|
+-------------------+-------------------+-----+
2021 |Mar |19282|
2021 |Jan |14757|
2021 |May |17130|
2021 |Feb |16935|
2021 |Apr |17592|
+-------------------+-------------------+-----+

In [0]:
# "Nuevos" datos de registros Junio y Julio de 2020 que no fueron incluidos en entrenamiento, pero serán utilizados para mostrar los resultados en el tablero
print((new_data.count(), len(new_data.columns)))
new_data = new_data.filter((col("AnoRegistroPeticion") == "2021") | ((col("AnoRegistroPeticion") == "2020") & (col("MesRegistroPeticion").isin(["Jun", "Jul", "Aug", "Sep", "Oct","Nov","Dec"]))))
print((new_data.count(), len(new_data.columns)))

date1=date.today()
date1=date1.strftime('%d/%m/%Y')
list1=list(date1.split('/'))
date1=list1[1]+list1[0]+list1[2]
# Guardar "nuevos" datos
file_location = "mnt/newdata/"+ date1 + ".csv"
print(file_location)
new_data.coalesce(1).write.csv(path=file_location, mode="overwrite", sep="|", header=True)

(85696, 72)
(85696, 72)
mnt/newdata/10132021.csv

# Parte de lectura de nuevos datos

In [0]:
#Automatización de lectura de archivo nuevo.
date1=date.today()
date1=date1.strftime('%d/%m/%Y')
list1=list(date1.split('/'))
date1=list1[1]+list1[0]+list1[2]

file_location = "/mnt/newdata/"+ date1 + ".csv"
print(file_location)
new_data = spark.read.csv(file_location, header=True, sep="|")

/mnt/newdata/10132021.csv

# Parte de limpieza de variables

In [0]:
# Ajustar nombres de columnas
names_columns = new_data.columns

names_columns = list(map(lambda string: \
                    normalize("NFKD", string).encode("ascii","ignore").decode("ascii") \
                        .replace('\n', ' ').replace(' ', '_').replace('(', '').replace(')', '') \
                    , names_columns))

new_data = new_data.toDF(*names_columns)

In [0]:
# Se crean listas de cómo debe ser casteada cada variable
columnas_longInt = ["NumeroPeticion"]
columnas_int = ['AnoRegistroPeticion',
                'CodDptoPeticionario', 'CodMunicipioPeticionario',
                'EdadPeticionario', 'CodDptoAfectado',
                'CodMunicipioAfectado', 'EdadAfectado']

columnas_float = []

columnas_date = ["fechaRegistroPeticion", "FechaNacAfectado"]

# Se define como columnas string todas aquellas que no están en ninguna de las anteriores
columnas_string = [x for x in new_data.columns if x not in columnas_int + columnas_float + columnas_date]

In [0]:
# Hay algunas columnas de fecha que traen las fechas en formato decimal de Excel (ej: 44256,8303819444 que representa 1/03/2021  19:55:45). Para ello se debe crear esta funcion convert_date_udf
def convert_date(x):
    mDt = datetime(1899, 12, 30) # Se genera fecha inicial que representa el valor 0 en Excel
    x = str(x)
    if x=="None":
      x = "1900-01-01 00:00:00"
    try:
      x = float(x)
      is_float = True
    except ValueError:
      is_float = False
    if is_float:
      dlt = mDt + timedelta(days=x)
      return dlt.strftime("%Y-%m-%d %H:%M:%S")
    else:
      return x
  
convert_date_udf = udf(lambda z: convert_date(z), StringType())

In [0]:
# Aquí vamos a castear todas las columnas de acuerdo con su tipo
for i in columnas_string:
  new_data = new_data.withColumn(i, new_data[i].cast(StringType()))
for i in columnas_longInt:
  new_data = new_data.withColumn(i, F.regexp_replace(i, ',', '.').cast(LongType()))
for i in columnas_int:
  new_data = new_data.withColumn(i, F.regexp_replace(i, ',', '.').cast(IntegerType()))
for i in columnas_float:
  new_data = new_data.withColumn(i, F.regexp_replace(i, ',', '.').cast(DoubleType()))
for i in columnas_date:
  # Aquí empleamos la función definida en la celda anterior para corregir fechas en formato numérico
  new_data = new_data.withColumn(i, convert_date_udf(i))
  # Ante posibles errores por variables de fecha, deben incluir el patrón de la fecha acá
  new_data = new_data.withColumn(i, \
                 when(new_data[i].like("_/_/__ __:__"), \
                      F.unix_timestamp(new_data[i], "d/M/yy H:mm").cast(TimestampType())). \
                 when(new_data[i].like("____-__-__ __:__:__"), \
                      F.unix_timestamp(new_data[i], "yyyy-MM-dd HH:mm:ss").cast(TimestampType())). \
                 when(new_data[i].like("____-__-__ __:__:__.___"), \
                      F.unix_timestamp(new_data[i], "yyyy-MM-dd HH:mm:ss.SSS").cast(TimestampType())). \
                 when(new_data[i].like("__/__/____ _:__"), \
                      F.unix_timestamp(new_data[i], "dd/MM/yyyy H:mm").cast(TimestampType())). \
                 otherwise( \
                      F.unix_timestamp(new_data[i], "dd/MM/yyyy HH:mm").cast(TimestampType())))
  # Fijar que cuando las fechas son del tipo "datetime.datetime(1900, 1, 1, 0, 0) - 1900-01-01 00:00:00" se debe poner como nulo
  new_data = new_data.withColumn(i, \
                 when(new_data[i]==datetime(1900, 1, 1, 0, 0), \
                      None). \
                 otherwise(new_data[i]))
  
# Como pueden ser variables importantes en el modelo, se extrae la hora y minuto de registro de la petición
new_data = new_data.withColumn("horaRegistroPeticion", F.hour(col("fechaRegistroPeticion")))
new_data = new_data.withColumn("minutoRegistroPeticion", F.minute(col("fechaRegistroPeticion")))

# Se convierte la variables de TimeStamp a Date. Así se eliminan las referencias a horas, minutos y segundos que tengan las variables y se ponen en un formato estándar para Spark
for i in columnas_date:
  new_data = new_data.withColumn(i, F.to_date(new_data[i]))

In [0]:
# Reemplazar todos los posibles valores nulos
def limpiar_nulos(df, debug=False):
    for contador, i in enumerate(list(df.columns)):
        if debug:
            print(contador, i)
        df = df.withColumn(i, when((col(i) == 'None') | \
            (col(i) == 'NULL') | \
            (col(i) == 'null') | \
            (col(i) == 'NaN') | \
            (F.trim(col(i)) == '') | \
            (col(i) == ' ' ) | \
            (col(i) == '#N/A' ) | \
            (col(i) == 'Sin Información' ) | \
            (col(i) == 'Sin Dato' ) | \
            (col(i) == '#¡REF!' ) | \
            (col(i) == 'VALOR NULO' ) | \
            (col(i) == 'SELECCIONE' ),
            None).otherwise(df[i]))
    return df
  
new_data = limpiar_nulos(new_data, debug=False)

In [0]:
# Creación de variable de PaisAfectado agrupado
new_data = new_data.na.fill("ND", subset="PaisResidenciaAfectado")

list_paises_10 = ['ARGENTINA', 'ARMENIA', 'COLOMBIA', 'ECUADOR', 'ESPAÑA', 'LIBANO', 'ND', 'PERU', 'PUERTO RICO', 'VENEZUELA']

list_paises_5 = ['ARMENIA', 'COLOMBIA', 'ESPAÑA', 'ND', 'VENEZUELA']

new_data = new_data.withColumn("PaisAfectado_10", when(col("PaisResidenciaAfectado").isin(list_paises_10), col("PaisResidenciaAfectado")).
                               otherwise("OTRO"))

new_data = new_data.withColumn("PaisAfectado_5",when(col("PaisResidenciaAfectado").isin(list_paises_5), col("PaisResidenciaAfectado")).
                               otherwise("OTRO"))

In [0]:
# Se recuperan las coordenadas de los centros zonales, así como el código de centro zonal direccionado y el código de departamento de la regional direccionada

# Primero renombramos algunos centros zonales que tienen diferente nombre entre new_data y el archivo con el listado de centros zonales
dicc_czcambios = {"CZ EL CARMEN DE BOLIVAR": "CZ CARMEN DE BOLIVAR", "CZ AGUSTIN CODAZZI": "CZ CODAZZI", "CZ 1 MONTERIA": "CZ MONTERIA 1", "CZ INTEGRAL NOROCCIDENTAL": "CZ NOROCCIDENTAL", "CZ INTEGRAL NORORIENTAL": "CZ NORORIENTAL", "CAIF COMUNA 13": "CZ ROSALES", "CZ SAN CRISTOBAL SUR": "CZ SAN CRISTOBAL", "CZ SAN JOSE DE GUAVIARE": "CZ SAN JOSE DEL GUAVIARE", "CZ SANTA MARTA 2": "CZ SANTA MARTA NORTE", "CZ SANTA MARTA 1": "CZ SANTA MARTA SUR", "CAIF COMUNA 8": "CZ SUR ORIENTE", "CZ YARIQUIES": "CZ YARIGUIES"}

new_data = new_data.na.replace(dicc_czcambios, subset="CZDireccionado")

# Se lee archivo que contiene esta información
file = "/mnt/data/Data/recuperacion_coordenadasCZ"
listado_coordcz = spark.read.parquet(file)

# Renombramos columnas en listado_coordcz para que no haya conflictos al hacer el join
listado_coordcz = listado_coordcz.withColumnRenamed("RegionalDireccionada", "RegionalDireccionada_join").withColumnRenamed("CZDireccionado", "CZDireccionado_join")

# Se establecen las condiciones para hacer el merge y así recuperar la información
cond = [new_data["RegionalDireccionada"] == listado_coordcz["RegionalDireccionada_join"],
        new_data["CZDireccionado"] == listado_coordcz["CZDireccionado_join"]]

# Se verifica que el número de filas no cambie antes y después del merge
print("Número de observaciones inicial: ", new_data.count())

# Se pega a new_data la nueva información 
new_data = new_data.join(listado_coordcz, cond, how='left')

print("Número de observaciones final: ", new_data.count())

#Se eliminan las dos columnas añadidas como llave de cruce que están repetidas
new_data = new_data.drop("RegionalDireccionada_join", "CZDireccionado_join")

Número de observaciones inicial: 85696
Número de observaciones final: 85696

In [0]:
# Ahora se adiciona la información de variables de contexto departamentales
# Leer archivo como pandas Dataframe
os.chdir(r'/dbfs//mnt/data/Data/')
file = "Variables_contexto_dptal.xlsx"
vardpto_contexto = pd.read_excel(file, sheet_name='importar', engine="openpyxl", converters={'Divipola_dpto1':str,'Divipola_dpto2':str})

# Por la anormalidad de la situación y las cifras del año 2020, para dicho año se tomarán los mismos valores observados en 2019
vars_contexto = ['tasa_vip_total', 'tasa_vif_total', "TasaFecunidad_15a19", "porc_rural", "vict_delsexual_total"]
for i in vars_contexto:
  nom_var_2019, nom_var_2020 = [i + "_2019", i + "_2020"]
  vardpto_contexto[nom_var_2020] = vardpto_contexto[nom_var_2019]

# Convertir el dataframe de formato wide a long
vardpto_contexto = pd.wide_to_long(vardpto_contexto, vars_contexto, i=['Divipola_dpto1', 'Divipola_dpto2', "porc_etnica_2018", "porc_indigena_2018"], j='agno_contexto', sep='_').reset_index()

#Convertir el archivo a Spark Dataframe
vardpto_contexto_sk = spark.createDataFrame(vardpto_contexto)

# Se hace el join o merge con la base de peticiones usando como llaves el código de departamento y que el año de la información de contexto sea del año anterior al de la peticion

#Creamos la columna de rezago del año de contexto
vardpto_contexto_sk = vardpto_contexto_sk.withColumn("agno_contexto_mas1", col("agno_contexto")+1)

# Se realiza el join usando la siguiente condición
cond = [new_data["CodDptoRegionalDireccionada"] == vardpto_contexto_sk["Divipola_dpto2"],
        new_data["AnoRegistroPeticion"] == vardpto_contexto_sk["agno_contexto_mas1"]]

new_data = new_data.join(vardpto_contexto_sk, cond, how='left')

# Se eliminan columnas innecesarias
new_data = new_data.drop("Divipola_dpto1", "Divipola_dpto2", "agno_contexto", "agno_contexto_mas1")

In [0]:
# Creamos una función que verificará que todas las homologaciones y limpieza de las variables se hagan correctamente

# Se verifica que las llaves tomadas en el diccionario contengan todas las categorías de la variable
# Para esto creamos una función que nos va a servir para todas las homologaciones/limpieza
def check_replace_llaves(df, var, diccionario):
    keys = list(diccionario.keys()) # Se genera una lista con las llaves del diccionario creado
    df_cuenta_valores = df.select(var).distinct().collect() # Se obtienn los valores únicos de la variable
    lista_valores = [str(row[var]) for row in df_cuenta_valores] # Se realiza una una lista con los valores únicos
    check = set(lista_valores).issubset(keys) # Check es True si las categorías de la variable están contenidas en el diccionario que creamos
    if check==True:
        df = df.na.replace(diccionario, subset=var) # Se usa el diccionario para hacer los reemplazos
        print(check, "= Todas las categorías de la variable {} están incluidas en el diccionario. \n \
        Se hizo el reemplazo de los valores de acuerdo al diccionario.".format(var))
    else:
        print(check, "No se hizo el reemplazo de los valores. \n \
        Aún falta incluir algunas categorías de la variable {} en el diccionario".format(var))
        print("Estas variables son: ", [x for x in lista_valores if x not in keys])
    return df

In [0]:
# Se define diccionario con categorías originales como llaves (keys) y homologaciones/limpieza como valores (values)
dicc_MotivoPeticion = {
    "Amenazado por grupos armados organizados al margen de la ley": "Desvinculados", 
    "Hechos Victimizantes en el Marco del Conflicto Armado": "Desvinculados",
    "Explotación Sexual Comercial \x13†ESCNNA": "ESCNNA", 
    "Pornografía Infantil - Violencia sexual - Explotación Sexual": "ESCNNA", 
    "Prostitución Infantil - Violencia sexual - Explotación Sexual": "ESCNNA",
    "Explotación Sexual Comercial \x96 ESCNNA" : "ESCNNA",
    "Maltrato a Niñ@ en Gestación": "Maltrato", "Maltrato Físico": "Maltrato", 
    "Maltrato físico": "Maltrato", "Maltrato por negligencia": "Maltrato", "Maltrato por Negligencia": "Maltrato", 
    "Maltrato psicológico": "Maltrato", "Maltrato Psicológico": "Maltrato", 
    "Violencia física, psicológica y/o negligencia": "Maltrato",
    "Abandono": "Abandono_Amenaza", "Amenazado": "Abandono_Amenaza",
    "Víctima de uso, porte, manipulación o lesión por pólvora": "Abandono_Amenaza", "Desnutrición": "Abandono_Amenaza",
    "Apoyo a madre gestante o lactante (menor o mayor de 18 años)": "PI_Vulnerable", 
    "Atención a madre gestante Sentencia C-355 de 2006": "PI_Vulnerable", 
    "Atención a niños y niñas hasta los tres (3) años de edad en establecimientos de reclusión de mujeres": "PI_Vulnerable",
    "Carece de representante legal": "NNA_solo", "Carece de Representante Legal": "NNA_solo", "Expósito": "NNA_solo", 
    "Extraviado": "NNA_solo", "Niños, niñas y adolescentes migrantes no acompañados": "NNA_solo","Falta de Responsables": "NNA_solo",
    "Conductas Sexualizadas entre menores de 14 años": "Conductas_Adolescentes", 
    "Conductas sexualizadas entre menores de 14 años": "Conductas_Adolescentes",
    "Consumo de sustancias psicoactivas": "Conductas_Adolescentes", "Consumo de Sustancias Psicoactivas": "Conductas_Adolescentes", 
    "Problemas de comportamiento": "Conductas_Adolescentes", "Problemas de Comportamiento": "Conductas_Adolescentes", 
    "Convivencia Educativa": "Conductas_Adolescentes",
    "Incumplimiento al régimen de visitas y custodia": "Otro", "Motivo SIUCE": "Otro", 
    "NNA Víctimas de desastres naturales u otras situaciones de emergencia": "Otro", 
    "Otorga consentimiento para la adopción": "Otro", "Por establecer": "Otro", "Solicitud vinculación programa de Discapacidad": "Otro",
    "Tenencia Irregular de Niño, Niña o Adolescente": "Otro",
    "Situación de Alta Permanencia en Calle": "Permanencia_Calle",  "Situación de Vida en calle": "Permanencia_Calle", 
    "Situación de vida en calle": "Permanencia_Calle", "Solicitud de refugio de niños, niñas y Adolescentes": "Permanencia_Calle", 
    "None": "ND", 
    "Situación de trabajo infantil": "TI", "Situación Trabajo Infantil (Sin frecuencia y tiempo determinado)": "TI", 
    "Trata con fines diferentes a la explotación sexual": "Trata", "Trata con fines sexuales": "Trata", 
    "Trata de Niños, Niñas y Adolescentes": "Trata", 
    "Abuso Sexual/Violencia Sexual": "V_sexual", "Violencia Sexual": "V_sexual"
}

# Ahora utilizamos la función creada para verificar que el diccionario contenga todas las categorías de la variable
new_data = check_replace_llaves(
    df = new_data, var = "MotivoPeticion", diccionario=dicc_MotivoPeticion
)
# Reemplazamos los valores nulos por categoría "ND"
new_data = new_data.na.fill(value="ND",subset=["MotivoPeticion"])

True = Todas las categorías de la variable MotivoPeticion están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.

In [0]:
# Ajuste de variales de Tipo de Documento
dicc_TipoDocumento = {
    "CEDULA DE CIUDADANIA": "CC", "CEDULA DE EXTRANJERIA": "CE, PASAPORTE, PEP O VISA", 
    "IDENTIDAD EXTRANJERA": "CE, PASAPORTE, PEP O VISA", "PASAPORTE": "CE, PASAPORTE, PEP O VISA", 
    "PERMISO DE PERMANENCIA": "CE, PASAPORTE, PEP O VISA", "TARJETA DE MOVILIDAD FRONTERIZA": "CE, PASAPORTE, PEP O VISA", 
    "TARJETA EXTRANJERIA": "CE, PASAPORTE, PEP O VISA", "VISA": "CE, PASAPORTE, PEP O VISA", 
    "CODIGO DANE DE IDENTIFICACIÓN ESTABLECIM": "DANE, NIT O RUT", 
    "NUMERO DE IDENTIFICACION TRIBUTARIA": "DANE, NIT O RUT", "REGISTRO UNICO TRIBUTARIO": "DANE, NIT O RUT", 
    "None": "ND", "NUMERO DE IDENTIFICACION NULO": "ND", "SIN DOCUMENTO": "ND", 
    "LIBRETA MILITAR": "ND", "NUMERO UNICO DE IDENTIFICACION PERSONAL": "ND", "OTRO": "ND", 
    "PARTIDA DE BAUTISMO": "RC o PB", "REGISTRO CIVIL": "RC o PB", "TARJETA DE IDENTIDAD": "TI"
}

# Aplicamos la función para la variable "TipoDocumentoPeticionario"
new_data = check_replace_llaves(
    df = new_data, var = "TipoDocumentoPeticionario", diccionario=dicc_TipoDocumento
)
# Reemplazamos los valores nulos por categoría "ND"
new_data = new_data.na.fill(value="ND",subset=["TipoDocumentoPeticionario"])

# Aplicamos la función para la variable "TipoDocumentoAfectado"
new_data = check_replace_llaves(
    df = new_data, var = "TipoDocumentoAfectado", diccionario=dicc_TipoDocumento
)
# Reemplazamos los valores nulos por categoría "ND"
new_data = new_data.na.fill(value="ND",subset=["TipoDocumentoAfectado"])

True = Todas las categorías de la variable TipoDocumentoPeticionario están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
True = Todas las categorías de la variable TipoDocumentoAfectado están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.

In [0]:
# Ajuste de variable EdadPeticionario
# Para la variable anterior se pondrán como nulos todos los valores que sean menores de 2 (niños que no pueden hablar o escribir no registrarían una petición) o mayores de 112 (valores poco frecuentes de la edad pero atípicos)
new_data = new_data.withColumn("EdadPeticionario", \
            when((new_data["EdadPeticionario"] < 2) | \
                (new_data["EdadPeticionario"] > 112), None) \
            .otherwise(new_data["EdadPeticionario"]))

deciles = [17.0, 24.0, 27.0, 30.0, 32.0, 35.0, 38.0, 42.0, 48.0]
new_data = new_data.withColumn("EdadPeticionario", \
            when(col("EdadPeticionario").isNull(), "ND"). \
            when((col("EdadPeticionario") <= deciles[0]), "17omenos"). \
            when((col("EdadPeticionario").between(deciles[0], deciles[1])), "18_24"). \
            when((col("EdadPeticionario").between(deciles[1], deciles[2])), "25_27"). \
            when((col("EdadPeticionario").between(deciles[2], deciles[3])), "28_30"). \
            when((col("EdadPeticionario").between(deciles[3], deciles[4])), "31_32"). \
            when((col("EdadPeticionario").between(deciles[4], deciles[5])), "33_35"). \
            when((col("EdadPeticionario").between(deciles[5], deciles[6])), "36_38"). \
            when((col("EdadPeticionario").between(deciles[6], deciles[7])), "39_42"). \
            when((col("EdadPeticionario").between(deciles[7], deciles[8])), "43_48"). \
            when((col("EdadPeticionario") > deciles[8]), "49omayor"). \
            otherwise("Error"))

In [0]:
# Limpieza edad afectado
TipoEdadErroneo = ["12", "Femenino", "Masculino", "No Aplica"]
new_data = new_data.withColumn("EdadAfectado_Anios", \
            when(new_data["TipoEdadAfectado"].isin(TipoEdadErroneo), None). \
            when(new_data["TipoEdadAfectado"] == "Años", col("EdadAfectado")). \
            when(new_data["TipoEdadAfectado"] == "Meses", F.round((col("EdadAfectado")/12),3)). \
            when(new_data["TipoEdadAfectado"] == "Días", F.round((col("EdadAfectado")/365),3)). \
            otherwise(None)) 

# Se reemplazan los valores que son exactamente 0 por Null, ya que no contienen información
new_data = new_data.withColumn("EdadAfectado_Anios", \
            when((col("EdadAfectado_Anios") == 0), None) \
            .otherwise(col("EdadAfectado_Anios")))

# Calcular años entre la fecha de registro de la petición y fecha de nacimiento del afectado
new_data = new_data.withColumn("EdadAfectado_Calculada", F.round(F.months_between(col("fechaRegistroPeticion"),col("FechaNacAfectado"))/F.lit(12),3))

# Si la edad calculada es mayor o igual a 5, redondear al entero más cercano. Si es negativa, dejar un valor nulo. Si es igual a 0, dejar un valor de 0.003 (1 día de nacimiento)
new_data = new_data.withColumn("EdadAfectado_Calculada",
                 when(col("EdadAfectado_Calculada") >= 5, F.round(col("EdadAfectado_Calculada"),0)). \
                 when(col("EdadAfectado_Calculada") < 0, None). \
                 when(col("EdadAfectado_Calculada") == 0, 0.003). \
                 otherwise(col("EdadAfectado_Calculada")))

# Reemplazar los valores nulos de "EdadAfectado_Anios" con los valores calculados en "EdadAfectado_Calculada"
new_data = new_data.withColumn('EdadAfectado_Anios', F.coalesce('EdadAfectado_Anios', 'EdadAfectado_Calculada'))

# Se encuentra que para edades muy altas hay multiples casos en que los que puede haber errores de digitación y es una mejor información la edad calculada. Haremos el reemplazo solo bajo ciertas condiciones
new_data = new_data.withColumn('EdadAfectado_Anios', \
                            when(((col('EdadAfectado_Anios') > 18) & \
                                (col('EdadAfectado_Anios') > col('EdadAfectado_Calculada')) & \
                                (col('EdadAfectado_Calculada').isNotNull()) & \
                                (col('EdadAfectado_Calculada') > 1) & \
                                (col('EdadAfectado_Calculada') < 21)), 
                                    col('EdadAfectado_Calculada')). \
                            otherwise(col("EdadAfectado_Anios")))

# Se encuentra que hay multiples registros con edades muy altas y no han sido corregidos con la técnica anterior. A todas las edades mayores a 98 se pondrán valores nulos
new_data = new_data.withColumn("EdadAfectado_Anios", \
            when((col("EdadAfectado_Anios") > 98), None) \
            .otherwise(col("EdadAfectado_Anios")))

# Se eliminan las variables de edad que ahora no son necesarias
new_data = new_data.drop("EdadAfectado_Calculada", "TipoEdadAfectado", "EdadAfectado")

# Se crea una variable de EdadAfectado_Agrupado, en la que se redondea la edad al entero menor (floor) y se crea el grupo "18_o_mas"
new_data = new_data.withColumn("EdadAfectado_Agrupado", when(col("EdadAfectado_Anios")<18, F.floor(col("EdadAfectado_Anios")).cast("string")).
                               when(col("EdadAfectado_Anios")>=18, "18_o_mas").
                               otherwise(col("EdadAfectado_Anios")))

In [0]:
# Se crea variable de EdadAfectado_Imputada, cuyos datos para ser imputados han sido guardados en el Storage Account. La imputación se realizará con base a las variables "TipoDocumentoAfectado" y "MotivoPeticion", ya que los valores que se imputarán se calcularon como la media por grupo en estas dos variables

file = "/mnt/data/Data/imputacion_EdadAfectado"
valores_imputacion = spark.read.parquet(file)

# Se realiza el join con el df principal
cond = [new_data["TipoDocumentoAfectado"] == valores_imputacion["TipoDocumentoAfectado2"],
        new_data["MotivoPeticion"] == valores_imputacion["MotivoPeticion2"]]

new_data = new_data.join(valores_imputacion, cond, how='left')

# Se crea columna con valores imputados
new_data = new_data.withColumn("EdadAfectado_Anios_Imputada", col("EdadAfectado_Anios"))
new_data = new_data.withColumn('EdadAfectado_Anios_Imputada', F.coalesce('EdadAfectado_Anios', 'Edad_Imputar'))

# Eliminar columnas innecesarias
new_data = new_data.drop("TipoDocumentoAfectado2", "MotivoPeticion2", "Edad_Imputar")

In [0]:
# Limpieza de variables de sexo
dicc_Sexo = {
    "Femenino": "F","Masculino": "M","Años": "ND",
    "No Aplica": "ND","No Reporta": "ND",
    "NO SE AUTORRECONOCE EN NINGUNO DE LOS ANTERIORES": "ND","None": "ND"
}

new_data = check_replace_llaves(
    df = new_data, var = "SexoPeticionario", diccionario=dicc_Sexo
)
# Para los valores nulos se requiere hacer un reemplazo adicional
new_data = new_data.na.fill("ND", subset="SexoPeticionario")

# Aplicamos la función para la variable "SexoAfectado"
new_data = check_replace_llaves(
    df = new_data, var = "SexoAfectado", diccionario=dicc_Sexo
)
# Para los valores nulos se requiere hacer un reemplazo adicional
new_data = new_data.na.fill("ND", subset="SexoAfectado")

True = Todas las categorías de la variable SexoPeticionario están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
True = Todas las categorías de la variable SexoAfectado están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.

In [0]:
# Variables de grupo étnico
dicc_GrupoEtnico = {
    "AFROCOLOMBIANO (A)": "AFROCOLOMBIANO (A)","COMUNIDAD NEGRA": "AFROCOLOMBIANO (A)",
    "NEGRO": "AFROCOLOMBIANO (A)","INDÍGENA": "INDÍGENA","Apuntador": "ND",
    "Masculino": "ND","N": "ND","N-D": "ND",
    "NELIDA IRIS SERRANO MENDOZA": "ND","NINGUNO": "ND",
    "NO SE AUTORRECONOCE EN NINGUNO DE LOS ANTERIORES": "ND",
    "None": "ND","Sin pertenencia étnica": "ND",
    "PALENQUERO": "PALENQUERO (A)","PALENQUERO (A)": "PALENQUERO (A)","RAIZAL": "RAIZAL",
    "san andres": "RAIZAL", "san andres ": "RAIZAL","ROM/GITANO": "ROM/GITANO"
}

# Aplicamos la función para la variable "GrupoEtnicoPeticionario"
new_data = check_replace_llaves(
    df = new_data, var = "GrupoEtnicoPeticionario", diccionario=dicc_GrupoEtnico
)
# Para los valores nulos se requiere hacer un reemplazo adicional
new_data = new_data.na.fill("ND", subset="GrupoEtnicoPeticionario")

# Aplicamos la función para la variable "GrupoEtnicoAfectado"
new_data = check_replace_llaves(
    df = new_data, var = "GrupoEtnicoAfectado", diccionario=dicc_GrupoEtnico
)
# Para los valores nulos se requiere hacer un reemplazo adicional
new_data = new_data.na.fill("ND", subset="GrupoEtnicoAfectado")

True = Todas las categorías de la variable GrupoEtnicoPeticionario están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
True = Todas las categorías de la variable GrupoEtnicoAfectado están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.

In [0]:
# Arreglo otras variables
#Se crea listado de variables que se puede homologar en conjunto valores inválidos o nulos
lista_var_homologar = ["DetalleZonaPeticionario", "PuebloIndigenaPeticionario", "PresentaDiscapacidadPeticionario", 
                       "CondicionDesplazamientoPeticionario", "HaSidoVictimaPeticionario", 
                       "DetalleZonAfectado", "PuebloIndigenaAfectado", "PresentaDiscapacidadAfectado", 
                       "CualDiscapacidadAfectado", "CondicionDesplazamientoAfectado"]

# Diccionario para limpiar valores nulos en estas variables
dicc_Nulos = {
    "N": None, "A": None, "No": None, "01/01/1900 0:00": None, "01/01/2004 0:00": None, "02/04/2007 0:00": None, 
    "03/03/1997 0:00": None, "07/11/2011 0:00": None, "19/09/2003 0:00": None, "2017-10-04 00:00:00.000": None, 
    "31/03/2001 0:00": None, "NO SE AUTORRECONOCE EN NINGUNO DE LOS ANTERIORES": None, 
    "Hijos de Padres que se Encuentran Privados de la Libertad por Orden Judicial": None, 
    "Omisión o negligencia": None, "Por Condiciones Especiales de Cuidadores": None
}

for i in lista_var_homologar:
    new_data = new_data.na.replace(dicc_Nulos, subset=i)
    new_data = new_data.na.fill("ND", subset=i)
    
# Hay unos pocos registros en los campos "DetalleZonaPeticionario" y "DetalleZonAfectado" que tienen valor "ND" o "Dispersa". El primero se homologa como "Urbana" al ser la categoría más frecuente y el segundo como "Rural"
new_data = new_data.na.replace({"ND": "Urbana", "Dispersa":"Rural"}, subset=["DetalleZonaPeticionario", "DetalleZonAfectado"])

In [0]:
# Creación de variable "TipoPeticionario"
# Limpieza de textos en campos de nombres de peticionario y afectado
@udf
def limpiar_col3(text):
  if text is not None:
    text = text.lower().strip().replace('.', '').replace('"', '').replace('ã‘', 'ñ').replace('  ', ' ')
    text = normalize('NFKD', text).encode("ascii","ignore").decode("ascii")
  return text

columnas_limpiar = ['NombresApellidosPeticionario', 'PrimerNombrePeticionario', 'SegundoNombrePeticionario', 'PrimerApellidoPeticionario', 'SegundoApellidoPeticionario', 'NombresApellidosAfectado', 'PrimerNombreAfectado', 'SegundoNombreAfectado', 'PrimerApellidoAfectado', 'SegundoApellidoAfectado', "BarrioVeredaAfectado", "DireccionAfectado", "ObsParticUbicaAfectado"]
for i in columnas_limpiar:
  new_data = new_data.withColumn(i, limpiar_col3(i))

In [0]:
new_data = new_data.withColumn("CategorizacionPeticionario",
                                                                 when((col("NombresApellidosPeticionario").contains("policia")) |
                                                                      (col("NombresApellidosPeticionario").contains("fiscal")) |
                                                                      (col("NombresApellidosPeticionario").contains("infancia y adolescencia")) |
                                                                      (col("NombresApellidosPeticionario").contains("polica")) |
                                                                      (col("NombresApellidosPeticionario").contains("polcia")) |
                                                                      (col("NombresApellidosPeticionario").contains("icalia")) |
                                                                      (col("NombresApellidosPeticionario").contains("caivas")) |
                                                                      (col("NombresApellidosPeticionario").contains("cavif")) |
                                                                      (col("NombresApellidosPeticionario").contains("cti")) |
                                                                      (col("NombresApellidosPeticionario").contains("cuerpo tecnico de investigacion")) |
                                                                      (col("NombresApellidosPeticionario").contains("investigacion criminal")) |
                                                                      (col("NombresApellidosPeticionario").contains("ponal")) |
                                                                      (col("NombresApellidosPeticionario").contains("inspeccion")) |
                                                                      (col("NombresApellidosPeticionario").contains("inspector")) |
                                                                      (col("NombresApellidosPeticionario").contains("tendente")) |
                                                                      (col("NombresApellidosPeticionario").contains("patruller")) |
                                                                      (col("NombresApellidosPeticionario").contains("ejercito")) |
                                                                      (col("NombresApellidosPeticionario").contains("militar")) |
                                                                      (col("NombresApellidosPeticionario").contains("batallon")) |
                                                                      (col("NombresApellidosPeticionario").contains("inpec")) |
                                                                      (col("NombresApellidosPeticionario").contains("sijin")),
                                                                      "Policia_Fiscalia").
                                                                 when((col("NombresApellidosPeticionario").contains("hospital")) |
                                                                      (col("NombresApellidosPeticionario").contains("hospial")) |
                                                                      (col("NombresApellidosPeticionario").contains("clinica")) |
                                                                      (col("NombresApellidosPeticionario").contains("empresa social del estado")) |
                                                                      (col("NombresApellidosPeticionario").contains("ips")) |
                                                                      (col("NombresApellidosPeticionario").contains("uss")) |
                                                                      (col("NombresApellidosPeticionario").contains("subred")) |
                                                                      (col("NombresApellidosPeticionario").contains("sub red")) |
                                                                      (col("NombresApellidosPeticionario").contains("red salud")) |
                                                                      (col("NombresApellidosPeticionario").contains("eps")) |
                                                                      (col("NombresApellidosPeticionario").contains("uci ")) |
                                                                      (col("NombresApellidosPeticionario").contains("de salud")) |
                                                                      (col("NombresApellidosPeticionario").contains("de la salud")) |
                                                                      (col("NombresApellidosPeticionario").contains("materno infantil")) |
                                                                      (col("NombresApellidosPeticionario").contains("cardio")) |
                                                                      (col("NombresApellidosPeticionario").contains("salud total")) |
                                                                      (col("NombresApellidosPeticionario").contains("clinico")) |
                                                                      (col("NombresApellidosPeticionario").contains("valle de lili")) |
                                                                      (col("NombresApellidosPeticionario").contains("valle del lili")) |
                                                                      (col("NombresApellidosPeticionario").contains("famisanar")) |
                                                                      (col("NombresApellidosPeticionario").contains("profamilia")) |
                                                                      (col("NombresApellidosPeticionario").contains("cuidado")) |
                                                                      (col("NombresApellidosPeticionario").contains("medic")) |
                                                                      (col("NombresApellidosPeticionario").contains("pediatr")) |
                                                                      (col("NombresApellidosPeticionario").contains("comfa")) |
                                                                      (col("NombresApellidosPeticionario").contains("meissen")) |
                                                                      (col("NombresApellidosPeticionario").contains("ese ")),
                                                                      "Hospital_Clinica").
                                                                 when((col("NombresApellidosPeticionario").contains("alcaldia")) |
                                                                      (col("NombresApellidosPeticionario").contains("municipio")) |
                                                                      (col("NombresApellidosPeticionario").contains("gobernacion")) |
                                                                      (col("NombresApellidosPeticionario").contains("idipron")) |
                                                                      (col("NombresApellidosPeticionario").contains("red unidos")) |
                                                                      (col("NombresApellidosPeticionario").contains("secretaria")) |
                                                                      (col("NombresApellidosPeticionario").contains("ministerio")) |
                                                                      (col("NombresApellidosPeticionario").contains("prosperidad social")) |
                                                                      (col("NombresApellidosPeticionario").contains("unidad nacional de proteccion")) |
                                                                      (col("NombresApellidosPeticionario").contains("migracion")) |
                                                                       (col("NombresApellidosPeticionario").contains("cancilleria")), 
                                                                       "Ejecutivo").
                                                                  when((col("NombresApellidosPeticionario").contains("centro zonal")) |
                                                                       (col("NombresApellidosPeticionario").contains("empi")) |
                                                                       (col("NombresApellidosPeticionario").contains("cespa")) |
                                                                       (col("NombresApellidosPeticionario").contains("unidad movil")) |
                                                                       (col("NombresApellidosPeticionario").contains("externado")) |
                                                                       (col("NombresApellidosPeticionario").contains("unidades moviles")) |
                                                                       (col("NombresApellidosPeticionario").contains("bienestar familiar")) |
                                                                       (col("NombresApellidosPeticionario").contains("defensor")) |
                                                                       (col("NombresApellidosPeticionario").contains("unidad de responsabilidad penal")) |
                                                                       (col("NombresApellidosPeticionario").contains("nutricional")) |
                                                                       (col("NombresApellidosPeticionario").contains("sede nacional")) |
                                                                       (col("NombresApellidosPeticionario").contains("regional")) |
                                                                       (col("NombresApellidosPeticionario").contains("restablecimiento de derechos")) |
                                                                       (col("NombresApellidosPeticionario").contains("icbf")) |
                                                                       (col("NombresApellidosPeticionario").contains("cdi")) |
                                                                       (col("NombresApellidosPeticionario").contains("cdi")) |
                                                                       (col("NombresApellidosPeticionario").startswith('cz ')), 
                                                                       "ICBF").
                                                                 when((col("NombresApellidosPeticionario").contains("comisari")) |
                                                                      (col("NombresApellidosPeticionario").contains("judicial")) |
                                                                      (col("NombresApellidosPeticionario").contains("justicia")) |
                                                                      (col("NombresApellidosPeticionario").contains("juzgado")),
                                                                     "Judicial").
                                                                 when((col("NombresApellidosPeticionario").contains("colegio")) |
                                                                      (col("NombresApellidosPeticionario").contains("escuela")) |
                                                                      (col("NombresApellidosPeticionario").contains("escolar")) |
                                                                      (col("NombresApellidosPeticionario").contains("liceo")) |
                                                                      (col("NombresApellidosPeticionario").contains("ied")) |
                                                                      (col("NombresApellidosPeticionario").contains(" ie ")) |
                                                                      (col("NombresApellidosPeticionario").startswith('ie ')) |
                                                                      (col("NombresApellidosPeticionario").startswith('iem ')) |
                                                                      (col("NombresApellidosPeticionario").contains("inem")) |
                                                                      (col("NombresApellidosPeticionario").contains("educaiva")) |
                                                                      (col("NombresApellidosPeticionario").contains("instituto tec")) |
                                                                      (col("NombresApellidosPeticionario").contains("tecnico")) |
                                                                      (col("NombresApellidosPeticionario").contains("rector")) |
                                                                      (col("NombresApellidosPeticionario").contains("educativa")),
                                                                     "Colegio").
                                                                 when((col("NombresApellidosPeticionario").contains("defensoria del pueblo")) |
                                                                      (col("NombresApellidosPeticionario").contains("procuraduria")) |
                                                                      (col("NombresApellidosPeticionario").contains("personeria")) |
                                                                      (col("NombresApellidosPeticionario").contains("de paso")) |
                                                                      (col("NombresApellidosPeticionario").contains("hogar paso")) |
                                                                      (col("NombresApellidosPeticionario").contains("notaria")),
                                                                     "MinisterioPublico_HogarDePaso").
                                                                 when((col("NombresApellidosPeticionario").contains("fundacion")) |
                                                                      (col("NombresApellidosPeticionario").contains("ong")) |
                                                                      (col("NombresApellidosPeticionario").contains("organizacion")) |
                                                                      (col("NombresApellidosPeticionario").contains("instituto")) |
                                                                      (col("NombresApellidosPeticionario").contains("institucion")) |
                                                                      (col("NombresApellidosPeticionario").contains("corpo")) |
                                                                      (col("NombresApellidosPeticionario").contains("corpro")) |
                                                                      (col("NombresApellidosPeticionario").contains("centro")) |
                                                                      (col("NombresApellidosPeticionario").contains("crecer en familia")) |
                                                                      (col("NombresApellidosPeticionario").contains("asociacion")),
                                                                     "Fundacion_ONG").
                                                                 when((col("NombresApellidosPeticionario").contains("anonimo")) |
                                                                      (col("NombresApellidosPeticionario").contains("anomino")) |
                                                                      (col("NombresApellidosPeticionario").contains("aninimo")) |
                                                                      (col("NombresApellidosPeticionario").contains("anomimo")) |
                                                                      (col("NombresApellidosPeticionario").contains("n o n i m o")) |
                                                                      (col("NombresApellidosPeticionario").contains("no report")) |
                                                                      (col("NombresApellidosPeticionario").contains("no refiere")) |
                                                                      (col("NombresApellidosPeticionario").contains("no registra")) |
                                                                      (col("NombresApellidosPeticionario").contains("nn nn")) |
                                                                      (col("NombresApellidosPeticionario").contains("nulo nulo")) |
                                                                      (col("NombresApellidosPeticionario").contains("sd ")) |
                                                                      (col("NombresApellidosPeticionario").contains("s-d")) |
                                                                      (col("NombresApellidosPeticionario").contains("xx")) |
                                                                      (col("NombresApellidosPeticionario").contains("x x")) |
                                                                      (col("NombresApellidosPeticionario").contains("nr nr")) |
                                                                      (col("NombresApellidosPeticionario").contains("na na")) |
                                                                      (col("NombresApellidosPeticionario").contains("n/a")) |
                                                                      (col("NombresApellidosPeticionario").contains("n n n n")) |
                                                                      (col("NombresApellidosPeticionario").contains("n n n")) |
                                                                      (col("NombresApellidosPeticionario").contains("nn nn")) |
                                                                      (col("NombresApellidosPeticionario").contains("nna nna")) |
                                                                      (col("NombresApellidosPeticionario").contains("anonima")) |
                                                                      (col("NombresApellidosPeticionario").contains("no aporta")) |
                                                                      (col("NombresApellidosPeticionario").contains("no aplica")) |
                                                                      (col("NombresApellidosPeticionario").contains("sin informacion")) |
                                                                      (col("NombresApellidosPeticionario").contains("123")) |
                                                                      (col("NombresApellidosPeticionario").contains("106")) |
                                                                      (col("NombresApellidosPeticionario").contains("linea de emergencia")) |
                                                                      (col("NombresApellidosPeticionario").contains("dato")), 
                                                                      "Anonimo").
                                                                 when((col("NombresApellidosPeticionario").isNull()) |
                                                                      (col("NombresApellidosPeticionario")=="n n") |
                                                                      (col("NombresApellidosPeticionario")=="n r") |
                                                                      (col("NombresApellidosPeticionario")=="nr") |
                                                                      (col("NombresApellidosPeticionario")=="nr nr") |
                                                                      (col("NombresApellidosPeticionario")=="na") |
                                                                      (col("NombresApellidosPeticionario")=="nn") | (col("NombresApellidosPeticionario")=="sd") | (col("NombresApellidosPeticionario")=="") |
                                                                      (col("NombresApellidosPeticionario")==" "), 
                                                                      "Anonimo").
                                                                 when((col("NombresApellidosPeticionario")==col("NombresApellidosAfectado")),
                                                                     "MismoAfectado").
                                                                 when((col("PrimerApellidoAfectado")==col("PrimerApellidoPeticionario")) |
                                                                      (col("PrimerApellidoAfectado")==col("SegundoApellidoPeticionario")) |
                                                                      (col("SegundoApellidoAfectado")==col("PrimerApellidoPeticionario")) |
                                                                      (col("SegundoApellidoAfectado")==col("SegundoApellidoPeticionario")),
                                                                      "Familiar").
                                                                 otherwise("Otro"))
new_data.groupby("CategorizacionPeticionario").count().sort("count").show()

+--------------------------+-----+
CategorizacionPeticionario|count|
+--------------------------+-----+
 MinisterioPublico...| 218|
 Judicial| 851|
 Ejecutivo| 980|
 Fundacion_ONG| 1102|
 ICBF| 1365|
 MismoAfectado| 1901|
 Colegio| 2239|
 Policia_Fiscalia| 6221|
 Hospital_Clinica|10399|
 Otro|14192|
 Anonimo|18460|
 Familiar|27768|
+--------------------------+-----+

In [0]:
# Creación de variable "SinSuficienteInfo_Afectado", que permite saber si hay suficiente información para poder identificar y localizar al afectado
sin_info = "an[oi][nm]i[mn][oa]|a n o n i m o|rep[or][ro]t[oae]|repota|no refiere|no registra|no sabe|no informa|no aporta|no aplica|nulo|no sabe|no aporta|no aplica|nn?nn|^sd ?|s-d| ?sd|xx|x x|^x$|^nr ?|^n r ?|^na ?|^nna ?|n ?n ?a ?|n/a|n/r|n n n|^n ?n$| nn$|^nr ?$| nr$|^n$|^no ?$|^nin[oa\@]s?|ningu?n[oa]|desconoc|sin iden|sin info|sin dato|sin mas dato|sin nombre|varios|menor|nas?citur|^cl 0 ?|^cl$|^calle$|^kr 0 ?|^av 0 ?|^centro$|sin n[uo]men|sin direccion|no la conoce|no tiene|no conoce|^0 ?|^ ?$|^-$"

new_data = new_data.withColumn("SinSuficienteInfo_Afectado",
                                                                 when((col("NombresApellidosAfectado").isNull()) |
                                                                      (col("NombresApellidosAfectado").rlike(sin_info)),
                                                                      "SinInfo").
                                                                 when(((col("BarrioVeredaAfectado").isNull()) & (col("DireccionAfectado").isNull()) & (col("ObsParticUbicaAfectado").isNull())) |
                                                                      ((col("BarrioVeredaAfectado").isNull()) & (col("DireccionAfectado").isNull()) & (col("ObsParticUbicaAfectado").rlike(sin_info))) |
                                                                      ((col("BarrioVeredaAfectado").isNull()) & (col("DireccionAfectado").isNull())) |
                                                                      ((col("BarrioVeredaAfectado").isNull()) & (col("DireccionAfectado").rlike(sin_info))) |
                                                                      ((col("BarrioVeredaAfectado").rlike(sin_info)) & (col("DireccionAfectado").isNull())) |
                                                                      ((col("BarrioVeredaAfectado").rlike(sin_info)) & (col("DireccionAfectado").rlike(sin_info))),
                                                                      "SinInfo").
                                                                 otherwise("ConInfo"))

In [0]:
# Se hace homologación de variable CanalRecepcion
dicc_canalatencion = {
  "Virtual": "Portal", "Instagram": "Redes Sociales", "FaceBook": "Redes Sociales", "Twitter": "Redes Sociales", "Buzón": "Escrito", "Teléfono Verde": "Telefónico Verbal"
}
new_data = new_data.na.fill("Telefónico Verbal", subset="CanalRecepcion")
new_data = new_data.na.replace(dicc_canalatencion, subset="CanalRecepcion")

# Predicción empleando modelo de redes neuronales

In [0]:
# Creamos conjunto de datos solo con variables que entrarán al modelo

columnas_mantener = ['AnoRegistroPeticion', 'MesRegistroPeticion', 'MotivoPeticion', 'CanalRecepcion','TipoDocumentoPeticionario', 'EdadPeticionario', 'SexoPeticionario', 'TipoDocumentoAfectado', 'SexoAfectado', 'GrupoEtnicoAfectado', 'PresentaDiscapacidadAfectado', 'CondicionDesplazamientoAfectado', 'horaRegistroPeticion', 'minutoRegistroPeticion', 'PaisAfectado_5', 'X', 'Y', 'porc_etnica_2018', 'porc_indigena_2018', 'tasa_vip_total', 'tasa_vif_total', 'TasaFecunidad_15a19', 'porc_rural', 'vict_delsexual_total', 'EdadAfectado_Anios_Imputada', 'DescripcionPeticion', 'TipoPeticion', 'CategorizacionPeticionario', 'SinSuficienteInfo_Afectado','DetalleZonAfectado','DetalleZonaPeticionario']

data_predict = new_data.select(columnas_mantener)

In [0]:
# Se verifica que no haya datos nulos en las variables incluidas en el modelo
pd.set_option("max_rows", 200)
count_null = data_predict.toHandy().isnull(ratio=False)
count_null

Out[32]: AnoRegistroPeticion 0
MesRegistroPeticion 0
MotivoPeticion 0
CanalRecepcion 0
TipoDocumentoPeticionario 0
EdadPeticionario 0
SexoPeticionario 0
TipoDocumentoAfectado 0
SexoAfectado 0
GrupoEtnicoAfectado 0
PresentaDiscapacidadAfectado 0
CondicionDesplazamientoAfectado 0
horaRegistroPeticion 0
minutoRegistroPeticion 0
PaisAfectado_5 0
X 2524
Y 2524
porc_etnica_2018 2524
porc_indigena_2018 2524
tasa_vip_total 2524
tasa_vif_total 2524
TasaFecunidad_15a19 2524
porc_rural 2524
vict_delsexual_total 2524
EdadAfectado_Anios_Imputada 0
DescripcionPeticion 1
TipoPeticion 0
CategorizacionPeticionario 0
SinSuficienteInfo_Afectado 0
DetalleZonAfectado 0
DetalleZonaPeticionario 0
Name: missing, dtype: int64

In [0]:
# Puede que en el campo DescripcionPeticion se encuentren registros vacíos, estos se llenarán solo con el texto "(Vacío)"
data_predict = data_predict.na.fill("(Vacío)", subset="DescripcionPeticion")

In [0]:
# Se convierte el Spark Dataframe en un Pandas Dataframe
data_predict_pd = data_predict.toPandas()

In [0]:
## Preprocesamiento de variables de texto a partir de pipeline guardado
# Separar la variable de texto
x_texto=pd.DataFrame(data_predict_pd.loc[:,"DescripcionPeticion"])

# Importar pipeline de limpieza de texto guardada con Mlfow
saved_text_pipeline = 'runs:/38239dc6bb4349d1b563ec5aa5928002/nn_text_pipeline'
text_pipeline = mlflow.sklearn.load_model(saved_text_pipeline)

# Hacer transformación de texto en padded_sequence con pipeline importado
padded_sequence = text_pipeline.transform(x_texto["DescripcionPeticion"])

In [0]:
####Separación de variables numéricas
numeric_feature_names = ['AnoRegistroPeticion', 'horaRegistroPeticion', 'minutoRegistroPeticion', 'X', 'Y', 'porc_etnica_2018', 'porc_indigena_2018', 'tasa_vip_total', 'tasa_vif_total', 'TasaFecunidad_15a19', 'porc_rural', 'vict_delsexual_total',  'EdadAfectado_Anios_Imputada']
numeric_features = data_predict_pd[numeric_feature_names]

# Importar pipeline de escalamiento de variables numéricas guardado con Mlfow
saved_num_pipeline = 'runs:/c7a9c2fa76b04a2dafa18f529b3966e1/nn_num_pipeline'
num_pipeline = mlflow.sklearn.load_model(saved_num_pipeline)

# Hacer transformación de variables numéricas con pipeline importado
numeric_features = num_pipeline.transform(numeric_features)
numeric_features_tf = tf.convert_to_tensor(numeric_features)

In [0]:
# Tratamiento de variables categóricas

# Separar variables categóricas
categorical_feature_names = ['MesRegistroPeticion', 'MotivoPeticion', 'CanalRecepcion','TipoDocumentoPeticionario', 'EdadPeticionario', 'SexoPeticionario',  'TipoDocumentoAfectado', 'SexoAfectado', 'GrupoEtnicoAfectado', 'PresentaDiscapacidadAfectado', 'CondicionDesplazamientoAfectado', 'PaisAfectado_5','TipoPeticion','CategorizacionPeticionario','SinSuficienteInfo_Afectado','DetalleZonAfectado','DetalleZonaPeticionario']
categorical_features=data_predict_pd[categorical_feature_names]
# One-hot encoding de las variables categóricas
categorical_features = pd.get_dummies(categorical_features)

# Ahora es necesario leer las mismas columnas que se crearon al hacer el preprocesamiento de redes neuronales, de forma que queden exactamente las mismas columnas
columnas_dummies = ['MesRegistroPeticion_Apr', 'MesRegistroPeticion_Aug', 'MesRegistroPeticion_Dec', 'MesRegistroPeticion_Feb', 'MesRegistroPeticion_Jan', 'MesRegistroPeticion_Jul', 'MesRegistroPeticion_Jun', 'MesRegistroPeticion_Mar', 'MesRegistroPeticion_May', 'MesRegistroPeticion_Nov', 'MesRegistroPeticion_Oct', 'MesRegistroPeticion_Sep', 'MotivoPeticion_Abandono_Amenaza', 'MotivoPeticion_Conductas_Adolescentes', 'MotivoPeticion_Desvinculados', 'MotivoPeticion_ESCNNA', 'MotivoPeticion_Maltrato', 'MotivoPeticion_ND', 'MotivoPeticion_NNA_solo', 'MotivoPeticion_Otro', 'MotivoPeticion_PI_Vulnerable', 'MotivoPeticion_Permanencia_Calle', 'MotivoPeticion_TI', 'MotivoPeticion_Trata', 'MotivoPeticion_V_sexual', 'CanalRecepcion_Chat', 'CanalRecepcion_Click to Call', 'CanalRecepcion_Correo Electrónico', 'CanalRecepcion_Escrito', 'CanalRecepcion_Portal', 'CanalRecepcion_Presencial Verbal', 'CanalRecepcion_Redes Sociales', 'CanalRecepcion_Telefónico Verbal', 'CanalRecepcion_VideoLlamada', 'CanalRecepcion_Web Service', 'CanalRecepcion_WhatsApp', 'TipoDocumentoPeticionario_CC', 'TipoDocumentoPeticionario_CE, PASAPORTE, PEP O VISA', 'TipoDocumentoPeticionario_DANE, NIT O RUT', 'TipoDocumentoPeticionario_ND', 'TipoDocumentoPeticionario_RC o PB', 'TipoDocumentoPeticionario_TI', 'EdadPeticionario_17omenos', 'EdadPeticionario_18_24', 'EdadPeticionario_25_27', 'EdadPeticionario_28_30', 'EdadPeticionario_31_32', 'EdadPeticionario_33_35', 'EdadPeticionario_36_38', 'EdadPeticionario_39_42', 'EdadPeticionario_43_48', 'EdadPeticionario_49omayor', 'EdadPeticionario_ND', 'SexoPeticionario_F', 'SexoPeticionario_M', 'SexoPeticionario_ND', 'TipoDocumentoAfectado_CC', 'TipoDocumentoAfectado_CE, PASAPORTE, PEP O VISA', 'TipoDocumentoAfectado_DANE, NIT O RUT', 'TipoDocumentoAfectado_ND', 'TipoDocumentoAfectado_RC o PB', 'TipoDocumentoAfectado_TI', 'SexoAfectado_F', 'SexoAfectado_M', 'SexoAfectado_ND', 'GrupoEtnicoAfectado_AFROCOLOMBIANO (A)', 'GrupoEtnicoAfectado_INDÍGENA', 'GrupoEtnicoAfectado_ND', 'GrupoEtnicoAfectado_PALENQUERO (A)', 'GrupoEtnicoAfectado_RAIZAL', 'GrupoEtnicoAfectado_ROM/GITANO', 'PresentaDiscapacidadAfectado_ND', 'PresentaDiscapacidadAfectado_Si', 'CondicionDesplazamientoAfectado_ND', 'CondicionDesplazamientoAfectado_S', 'PaisAfectado_5_ARMENIA', 'PaisAfectado_5_COLOMBIA', 'PaisAfectado_5_ESPAÑA', 'PaisAfectado_5_ND', 'PaisAfectado_5_OTRO', 'PaisAfectado_5_VENEZUELA', 'TipoPeticion_Reporte de Amenaza o Vulneración de derechos', 'TipoPeticion_Solicitud de Restablecimiento de Derechos (SRD)', 'CategorizacionPeticionario_Anonimo', 'CategorizacionPeticionario_Colegio', 'CategorizacionPeticionario_Ejecutivo', 'CategorizacionPeticionario_Familiar', 'CategorizacionPeticionario_Fundacion_ONG', 'CategorizacionPeticionario_Hospital_Clinica', 'CategorizacionPeticionario_ICBF', 'CategorizacionPeticionario_Judicial', 'CategorizacionPeticionario_MinisterioPublico_HogarDePaso', 'CategorizacionPeticionario_MismoAfectado', 'CategorizacionPeticionario_Otro', 'CategorizacionPeticionario_Policia_Fiscalia', 'SinSuficienteInfo_Afectado_ConInfo', 'SinSuficienteInfo_Afectado_SinInfo', 'DetalleZonAfectado_Rural', 'DetalleZonAfectado_Urbana', 'DetalleZonaPeticionario_Rural', 'DetalleZonaPeticionario_Urbana']

# Reindexar el one-hot encoding de forma que queden con las mismas columas que el entrenamiento
categorical_features = categorical_features.reindex(columns = columnas_dummies, fill_value=0)
categorical_features_tf=tf.convert_to_tensor(categorical_features)

In [0]:
##El siguiente código se usa para hacer load a un modelo preentrenado
path = "/dbfs/mnt/sample/Red1/data/model"
red1 = tf.keras.models.load_model(path)

In [0]:
predictions_new_data = red1.predict([padded_sequence,numeric_features, categorical_features], batch_size=32)

In [0]:
# Recuperar categoría predicha y las probabilidades asociadas a cada categoría
data_preds = pd.DataFrame({"Triage_Prediccion5categorias": np.argmax(predictions_new_data, axis=1), "ProbabilidadDePrediccion_Falsa":predictions_new_data[:,0], "ProbabilidadDePrediccion_SinDefinirOFallida":predictions_new_data[:,1], "ProbabilidadDePrediccion_VerdaderaNoPARD":predictions_new_data[:,2], "ProbabilidadDePrediccion_VerdaderaPARDNoInstitucional":predictions_new_data[:,3], "ProbabilidadDePrediccion_VerdaderaPARDInstitucional":predictions_new_data[:,4]})

# Cambiar las categorias de la variable predicha de número a categorias
dicc_varoobj = {"Prediccion Falsa": 0, "Prediccion SinDefinir o Fallida":1, "Prediccion Verdadera No PARD":2, "Prediccion Verdadera PARD No Institucional":3, "Prediccion Verdadera PARD Institucional":4}
dicc_val_var = {v: k for k, v in dicc_varoobj.items()}
data_preds["Triage_Prediccion5categorias"] = [dicc_val_var.get(item,item)  for item in data_preds["Triage_Prediccion5categorias"]]

# Agrupación de categorias predichas en Verdaderas / Falsa o Sin Definir
data_preds["Triage_Prediccion2categorias"] = np.where(
  data_preds["Triage_Prediccion5categorias"].isin(["Prediccion Falsa", "Prediccion SinDefinir o Fallida"]), "Prediccion Sin Definir, Fallida o Falsa", "Prediccion Verdadera")

# Reordenar columnas
data_preds = data_preds.reindex(columns=["Triage_Prediccion2categorias", "Triage_Prediccion5categorias", "ProbabilidadDePrediccion_Falsa", "ProbabilidadDePrediccion_SinDefinirOFallida", "ProbabilidadDePrediccion_VerdaderaNoPARD", "ProbabilidadDePrediccion_VerdaderaPARDNoInstitucional", "ProbabilidadDePrediccion_VerdaderaPARDInstitucional"])

# Cambiar formato de probabilidades para que en vez de número entre 0 y 1 se muestre como porcentaje entre 0 y 100
for i in ["ProbabilidadDePrediccion_Falsa", "ProbabilidadDePrediccion_SinDefinirOFallida", "ProbabilidadDePrediccion_VerdaderaNoPARD", "ProbabilidadDePrediccion_VerdaderaPARDNoInstitucional", "ProbabilidadDePrediccion_VerdaderaPARDInstitucional"]:
  data_preds[i] = pd.Series(["{0:.1f}%".format(val * 100) for val in data_preds[i]], index = data_preds.index)

# Añadir predicción a nuevos datos limpios

In [0]:
# Pasar anterior dataframe de pandas a dataframe de Spark
data_preds_spark = spark.createDataFrame(data_preds) 
#add 'sequential' index and join both dataframe to get the final result
new_data = new_data.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
data_preds_spark = data_preds_spark.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

final_df = new_data.join(data_preds_spark, new_data.row_idx == data_preds_spark.row_idx).drop("row_idx")

In [0]:
spa_stopwords = stopwords.words('spanish')
contexto_stopwords = lista_stopwords()
nombres = lista_nombres()
apellidos = lista_apellidos()
todos_geo = lista_geo_colombia()
spa_stopwords.extend(contexto_stopwords + nombres[0] +  apellidos[0] + todos_geo[0])

# Luego de analizar el texto se encuentra que aún hay una larga lista de nombres y apellidos que debe ser removida. Esta lista se encuentra en un archivo excel, el cual se convertirá a una lista para agregarlo al listado de stopwords anterior "spa_stopwords"
adicional_stopwords = pd.read_excel(r'/dbfs//mnt/basecruzada/listado nombres para eliminar.xlsx', sheet_name='listado nombres para eliminar', engine="openpyxl")["word"].to_list()
spa_stopwords.extend(adicional_stopwords)

# Limpiar texto de la lista de stopwords
def limpiar_col_list(text):
  if text is not None:
    text = normalize('NFKD', text).encode("ascii","ignore").decode("ascii")
    text = text.lower().strip().replace('.', '').replace('"', '').replace('ã‘', 'ñ').replace('  ', ' ').replace('hotmail', '').replace('gmail', '').replace('outlook', '').replace('yahoo', '').replace('icbfgovco', '').replace('ano', 'año').replace('senor', 'señor').replace('nino', 'niño').replace('nina', 'niña').replace('puno', 'puño')
  return text

spa_stopwords = [limpiar_col_list(x) for x in spa_stopwords]

# Finalmente, convertir la lista a set para que sea más rápida la ejecución
set_spa_stopwords = set(spa_stopwords)

In [0]:
@udf
def limpiar_col_descripcion(text):
  if text is not None:
    text = normalize('NFKD', text).encode("ascii","ignore").decode("ascii")
    text = text.lower().strip().replace('.', '').replace('"', '').replace('ã‘', 'ñ').replace('  ', ' ').replace('hotmail', '').replace('gmail', '').replace('outlook', '').replace('yahoo', '').replace('icbfgovco', '').replace('ano', 'año').replace('senor', 'señor').replace('nino', 'niño').replace('nina', 'niña').replace('puno', 'puño').replace('companer', 'compañer')
  return text

In [0]:
# Se crea una nueva columna llamada "DescripcionPeticion_Limpia" que limpia a partir de la función anterior el campo de DescripcionPeticion
final_df = final_df.withColumn("DescripcionPeticion_Limpia", limpiar_col_descripcion("DescripcionPeticion"))
final_df.cache()
final_df.count()

Out[44]: 85696

In [0]:
# Mantener solo las palabras frecuentes (que aparezcan por lo menos 50 veces en el texto) y con un largo de por lo menos 2 letras

# El archivo parquet que aquí se lee fue generado previamente usando este mismo conjunto de datos
freq_50 = pd.read_parquet(r'/dbfs//mnt/basecruzada/word_frequencies_parquet')
freq_50["largo"] =  freq_50["word"].map(len) # Calcular el largo de la palabra
freq_50 = freq_50.loc[(freq_50["count"]>49) & (freq_50["largo"]>1)] # Dejar solo aquellas palabras con cierta frecuencia (por lo menos 50 ocurrencias) y de más de una letra
list_freq_50 = freq_50["word"].to_list() # Convertir el dataframe a una lista de Python
set_freq_50 = set(list_freq_50) # Convertir a un set para hacer más rápida la ejecución
print(len(list_freq_50)) # El resultado es 27.255, es decir, solo se dejarán las 27.255 palabras más frecuentes

27255

In [0]:
@udf
def mantener_palabras(text):
  if text is not None:
    items = [item for item in text.split(" ") if item in set_freq_50 if item not in set_spa_stopwords]
    text = ' '.join(items)
  return text

In [0]:
final_df = final_df.withColumn("DescripcionPeticion_Limpia", mantener_palabras("DescripcionPeticion_Limpia"))
final_df.cache()
final_df.count()

Out[47]: 85696

In [0]:
# Finalmente se escribe esta tabla en la instancia SQL para que pueda ser consumida en el tablero de visualización
jdbcHostname = dbutils.secrets.get('blobsecret','SQLHostname')
jdbcDatabase = dbutils.secrets.get('blobsecret','SQLDatabase')
jdbcPort = dbutils.secrets.get('blobsecret','SQLport')
jdbcuser = dbutils.secrets.get('blobsecret','SQLusername')
jdbcpass = dbutils.secrets.get('blobsecret','SQLpassword')
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, jdbcuser, jdbcpass)

final_df.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("url", jdbcUrl).option("dbtable", 'TABLERO_DEFENSORES_PREDICCIONES').option("user", jdbcuser).option("password", jdbcpass).save()

In [0]:
# También se escribe en un archivo parquet para poder descargar y tener el backup
final_df.coalesce(1).write.mode('overwrite').parquet("mnt/basecruzada/DatosFinales_TableroDefensores_Predicciones_parquet")